# Crypto Challenge Set 1

https://cryptopals.com/sets/1

## 1. Convert hex to base64

In [1]:
import codecs

def hex2base64(s):
    decode_hex = codecs.getdecoder("hex_codec")
    encode_b64 = codecs.getencoder("base64")
    return encode_b64(decode_hex(s)[0])[0]

In [2]:
s = r"49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
# SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
hex2base64(s)

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t\n'

In [4]:
# binascii library is a better/friendlier option!

import binascii

s = r"49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"

print(binascii.b2a_base64(binascii.unhexlify(s)))
print("".join(chr(n) for n in binascii.unhexlify(s)))

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t\n'
I'm killing your brain like a poisonous mushroom


## 2. Fixed XOR

In [5]:
def fixedXOR(a,b):
    return hex(a^b)

In [6]:
a = 0x1c0111001f010100061a024b53535009181c
b = 0x686974207468652062756c6c277320657965
# 746865206b696420646f6e277420706c6179
fixedXOR(a,b)

'0x746865206b696420646f6e277420706c6179'

In [8]:
import binascii

a = '1c0111001f010100061a024b53535009181c'
b = '686974207468652062756c6c277320657965'

print( "".join([chr(an^bn) for an,bn in zip(binascii.unhexlify(a),binascii.unhexlify(b))]) )
print( "".join([hex(an^bn)[2:] for an,bn in zip(binascii.unhexlify(a),binascii.unhexlify(b))]) )

the kid don't play
746865206b696420646f6e277420706c6179


## 3. Single-byte XOR cipher

https://cryptopals.com/sets/1/challenges/3

In [12]:
import binascii

# using letter frequencies to compute sentence score
# https://en.wikipedia.org/wiki/Letter_frequency#Relative_frequencies_of_letters_in_the_English_language

letterFreq = {
  " ": 15,
  "e": 12.702,
  "t": 9.056,
  "a": 8.167,
  "o": 7.507,
  "i": 6.966,
  "n": 6.749,
  "s": 6.327,
  "h": 6.094,
  "r": 5.987,
  "d": 4.253
}

def sentenceScore(s):
    return sum([ letterFreq[c] for c in s if c in letterFreq.keys() ])

# If text has been XOR'd against a single character, there are 256 possible keys (2 hex digits)

def singleXORdecode(s):
    # compute all probable decoded strings according to keys
    strings = [ ''.join( chr(n^k) for n in s ) for k in range(256) ]
    # choose decoded message according to letter frequency score
    decoded = max(strings, key=sentenceScore)
    key = chr(strings.index(decoded))
    return key,decoded

In [16]:
s = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"

sbin = binascii.unhexlify(s) 
key, decoded = singleXORdecode(sbin)
print(key)
print(decoded)

X
Cooking MC's like a pound of bacon


## 4. Detect single-character XOR

https://cryptopals.com/sets/1/challenges/4

In [17]:
import binascii

strings = []
with open("input1/4.txt") as f:
    for l in f.readlines():
        s = l.strip("\n")
        sbin = binascii.unhexlify(s) 
        _,best = singleXORdecode(sbin)
        strings.append(best)

decoded = max(strings, key=sentenceScore)
print(decoded)

Now that the party is jumping



## 5. Implement repeating-key XOR

In [18]:
from binascii import hexlify

def repeatingKeyXOR(key, string):
    i = 0
    enc = []
    for c in string:
        enc.append( ord(c)^ord(key[i]) ) # XOR with current key letter
        i = (i+1)%len(key) # wrap round key lenght
    return hexlify( bytearray(enc) )

string = "Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"
key = 'ICE'

encrypted = repeatingKeyXOR(key, string)
print(encrypted)

b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'


## 6. Break repeating-key XOR

https://cryptopals.com/sets/1/challenges/6

In [19]:
def HammingDistance(b1,b2):
    # count differrent bits
    countdiffbits = 0
    for B1,B2 in zip(b1, b2):
        bindiff = bin(B1^B2) # different bits in binary format
        countdiffbits += bindiff.count("1") # count occurrences of "1"
    return countdiffbits

s1 = "this is a test"
s2 = "wokka wokka!!!"

# convert standard strings into byte arrays    
b1 = bytearray([ord(c) for c in s1])
b2 = bytearray([ord(c) for c in s2])

HammingDistance(b1,b2)

37

In [30]:
from binascii import a2b_base64
from itertools import combinations

def guessRepKeyXOR(s):
    # convert base64 text to binary
    b = a2b_base64(s)

    # guess keysize by testing testing several values and choosing that giving the smallest
    # normalised Hamming distance on bocks of that size
    keys = []
    for ks in range(2,41):
        # compute normalised Hamming distance between all combinations of blocks of size ks
        nbloc = len(b)//ks
        blocks = [ b[j*ks:(j+1)*ks] for j in range(nbloc) ]
        ndave = 0
        ncomb = 0
        for c in combinations(blocks,2):            
            ndave += HammingDistance(c[0],c[1])
            ncomb += 1
        ndave /= ncomb*ks
        keys.append((ks,ndave))

    # choose keysize as that giving smallest average Hamming distance between neigbouring blocks
    keys = sorted(keys,key=lambda x: x[1])
    keysize = keys[0][0]
    print("Guessed KEYSIZE =",keysize)

    # Break the ciphertext into blocks of KEYSIZE length, then transpose the blocks. Make a block that is the
    # first byte of every block, and a block that is the second byte of every block, and so on.
    # This is because each corresponding byte in all blocks has been encrypted with the same key character,
    # thus I can try to guess the key character it using the single-character XOR attach implemented at point 4.
    nblocks = len(b)//keysize # I'm skipping the last part of the cypher, I could maybe pad it to use the last block
    blocks = []
    for k in range(keysize):
        tblock = []
        for i in range(nblocks):
            tblock.append(b[k+keysize*i])
        blocks.append(tblock)

    # Solve each block as if it was single-character XOR, recompose the key!
    key = ""
    for b in blocks:
        strings = [ ''.join( chr(n^k) for n in b ) for k in range(256) ]
        decoded = max(strings, key=sentenceScore)
        key += chr(strings.index(decoded))
    print("Guessed KEY =",key)
    return key

In [31]:
with open("input1/6.txt") as f:
    s = f.read().replace("\n","")
    
key = guessRepKeyXOR(s)

Guessed KEYSIZE = 29
Guessed KEY = Terminator X: Bring the noise


In [34]:
# XOR is commutative: I can decode with the same algorithm used to encode with repeating-key XOR

def decodeRepKeyXOR(s,key):
    b = a2b_base64(s)
    i = 0
    enc = []
    for c in b:
        enc.append( c^ord(key[i]) ) # XOR with current key letter
        i = (i+1)%len(key) # wrap round key lenght
    return "".join(chr(i) for i in enc)

In [35]:
dec = decodeRepKeyXOR(s,key)
print(dec)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## 7. AES in ECB mode

https://cryptopals.com/sets/1/challenges/7

Cypher encrypted via AES-128 in ECB mode under the key "YELLOW SUBMARINE"

Using PyCryptoDone library:

https://www.pycryptodome.org/en/latest/

`pip install pycryptodome`

In [61]:
from binascii import a2b_base64
from Crypto.Cipher import AES

key = "YELLOW SUBMARINE" 

with open("input1/7.txt") as f:
    cipher = a2b_base64( f.read().strip("\n") )

aes = AES.new(key.encode(), AES.MODE_ECB) # key must be binary, encode() takes care of that
plain = aes.decrypt(cipher).decode() # the output is binary, decode() converts it to regular string
print(plain)

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## 8. Detect AES in ECB mode

https://cryptopals.com/sets/1/challenges/8

https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Electronic_codebook_(ECB)

Remember that the problem with ECB is that it is stateless and deterministic; the same 16 byte plaintext block will always produce the same 16 byte ciphertext.

I can split the ciphers in such blocks and look for possible repetitions.

In [3]:
from binascii import a2b_base64

with open("input1/8.txt") as f:
    ciphers = [ a2b_base64(l.strip("\n")) for l in f.readlines() ]

blocksize = 16
l = 1
for c in ciphers:
    blocks = [ c[i:i+blocksize] for i in range(0,len(c),blocksize) ]
    rep = len(blocks) - len(set(blocks)) # python sets ignore repetitions
    if rep:
        print("Cipher at line {} has {} block repetitions".format(l,rep))
    l += 1

Cipher at line 133 has 3 block repetitions
